In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from cnn_utils import train, accuracy

In [2]:
seed = 123
dtype = torch.float32
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
torch.set_default_device(device)
torch.manual_seed(seed)
generator = torch.Generator(device=device).manual_seed(seed)

In [3]:
# from datasets import load_dataset
# ds = load_dataset("slegroux/tiny-imagenet-200-clean")

# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# def transform_ds(batch):
#     batch["image"] = [transform(img) for img in batch["image"]]
#     return batch

# dataset = ds.with_transform(lambda x: transform_ds(x))

# train_dataset = dataset["train"]
# test_dataset = dataset["test"]
# val_dataset = dataset['validation']

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, generator=generator)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, generator=generator)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, generator=generator)

In [4]:
dataset = torchvision.datasets.FashionMNIST(root = './cnn_dataset',
                                               train = True,
                                               transform = transforms.Compose([
                                                   transforms.Resize((224, 224)),
                                                   transforms.ToTensor(),
                                               ]),
                                               download = True)
test_dataset = torchvision.datasets.FashionMNIST(root = './cnn_dataset',
                                               train = False,
                                               transform = transforms.Compose([
                                                   transforms.Resize((224, 224)),
                                                   transforms.ToTensor(),
                                               ]),
                                               download = True)

total_size = len(dataset)
train_size = int(total_size * 0.8)
val_size = int(total_size * 0.2)

train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, generator=generator)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, generator=generator)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, generator=generator)

In [5]:
class AlexNet(nn.Module):

    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.layers = nn.Sequential(
            nn.LazyConv2d(96, kernel_size=11, stride=4, padding=1),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LazyConv2d(256, kernel_size=5, padding=2), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LazyConv2d(384, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.LazyConv2d(384, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.LazyConv2d(256, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2), 
            nn.Flatten(),
            nn.LazyLinear(4096), nn.ReLU(), 
            nn.Dropout(p=0.5),
            nn.LazyLinear(4096), nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.LazyLinear(num_classes)
        )

    def forward(self, x):
        return self.layers(x)

In [6]:
output_size = 10
learning_rate = 1e-3

model = AlexNet(output_size)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
config = {
    "epoch" : 20,
    "print_per_epoch": 1,
    "device": device
}

train(model, criterion, optimizer, train_loader, val_loader, config)
acc = accuracy(model, test_loader, device)

print('ACC {}'.format(acc))

LOSS train 0.6034610393817226 valid 0.3610554906129837
LOSS train 0.3395211027910312 valid 0.3240429779291153
LOSS train 0.29169135929147405 valid 0.26630316814780236
LOSS train 0.2642030742950737 valid 0.2707809422214826
LOSS train 0.2424084764290601 valid 0.25179985917111236
LOSS train 0.2225952960724632 valid 0.24312122026085853
LOSS train 0.21314185327912372 valid 0.2504298675159613
LOSS train 0.19616748427599667 valid 0.24498437702655793
LOSS train 0.188476123607407 valid 0.2465942451407512
LOSS train 0.1779531557255735 valid 0.24714284587651492
LOSS train 0.17037369275527695 valid 0.23077028650417925
LOSS train 0.16452303640979032 valid 0.24053801298886537
LOSS train 0.15673319793174353 valid 0.23364908490578334
LOSS train 0.15039672307825336 valid 0.2563107480679949
LOSS train 0.14203970025091742 valid 0.2765418252547582
LOSS train 0.13898530744807794 valid 0.2457275699774424
LOSS train 0.12946846306494747 valid 0.26464771291986106
LOSS train 0.12744115247391163 valid 0.28621839